In [2]:
from pull_match_list import load_match_list
import pandas as pd


In [3]:
df = load_match_list()

In [4]:
df.columns

Index(['Champion', 'Result', 'Score', 'Build', 'Duration', 'Date', 'Game',
       'Tournament', 'player', 'gg_id', 'season', 'duration_minutes'],
      dtype='object')

In [5]:
df1 = df[((df["player"] == "ON") | (df["player"] == "meiko")| (df["player"] == "Crisp") | (df["player"] == "Hang")) & (df["season"] == "S14")]

In [6]:
df1 = df1[df1["Tournament"] != "Demacia Cup 2023"]

In [7]:
df1 = df1[["Champion","Score","Result","Date","Game","player","Tournament","duration_minutes"]]

In [8]:
from daily_report_functions import calc_kda
df1 = calc_kda(df1 ,"Score","")

In [9]:
df1 = df1.drop(columns = ["Score"])

In [10]:
match_result = df1.groupby(["player","Date","Game"]).apply(
    lambda group: pd.Series({
        "n_Defeat": (group["Result"] == "Defeat").sum(),
        "n_Victory": (group["Result"] == "Victory").sum(),
    })
).reset_index()

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_30196\3231435481.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  match_result = df1.groupby(["player","Date","Game"]).apply(


In [11]:
import numpy as np

# match_result["match_result"] = np.where(
#     match_result["n_Victory"] > match_result["n_Defeat"], "Victory",
#     np.where(match_result["n_Defeat"] > match_result["n_Victory"], "Defeat", "Other")
# )

match_result["match_result"] = match_result.apply(
    lambda row: "Victory" if row["n_Victory"] > row["n_Defeat"]
    else "Defeat" if row["n_Defeat"] > row["n_Victory"]
    else "Other",
    axis=1
)


In [12]:
match_result[match_result["match_result"] == "Other"]

,player,Date,Game,n_Defeat,n_Victory,match_result


In [13]:
match_result["match_type"] = match_result.apply(
    lambda row: "BO1" if max(row["n_Victory"], row["n_Defeat"]) == 1
    else "BO3" if max(row["n_Victory"], row["n_Defeat"]) == 2
    else "BO5" if max(row["n_Victory"], row["n_Defeat"]) == 3
    else "Other",
    axis=1
)


In [14]:
match_result[match_result["match_type"] == "Other"]

,player,Date,Game,n_Defeat,n_Victory,match_result,match_type


1. 表：每位选手的大概情况：总局数，胜率，场均时长，胜场场均时长，败场场均时长，平均KDA（分开来再merge）
2. 表：每个英雄每位选手的胜率和局数
3. 分类柱状图：每个英雄选手的胜率和局数
4. scatter：每位选手每小局的KDA，Duration（三张小图 vs 一张大图）
5. 折线统计图，每位选手每个月的胜局
6. 箱式，barplot with std：kda分布
7. heatmap
8. 饼状图：
9. 直方图vs密度图
10. 堆叠式面积图
11. 气泡？

In [15]:
import matplotlib.pyplot as plt 
import seaborn as sns 


In [16]:
match_result.head(3)

,player,Date,Game,n_Defeat,n_Victory,match_result,match_type
0,Crisp,2024-01-25,WBG vs Rare Atom,0,2,Victory,BO3
1,Crisp,2024-01-28,TT vs WBG,0,2,Victory,BO3
2,Crisp,2024-02-01,LNG Esports vs WBG,2,1,Defeat,BO3


In [17]:
df1.head(3)

,Champion,Result,Date,Game,player,Tournament,duration_minutes,K,D,A
4938,Maokai,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,27.750000,0,5,6
4939,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,31.616667,1,3,9
4940,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,39.283333,0,3,8


### 1. 表:局数/胜局数 vs Match_type


In [18]:
df_total = pd.merge(df1, match_result[["player","Date","Game","match_result","match_type"]], on = ["player","Date","Game"])

In [19]:
df_total.to_csv("sup_df.csv")

In [20]:
df_total.head(3)

,Champion,Result,Date,Game,player,Tournament,duration_minutes,K,D,A,match_result,match_type
0,Maokai,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,27.750000,0,5,6,Defeat,BO5
1,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,31.616667,1,3,9,Defeat,BO5
2,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,39.283333,0,3,8,Defeat,BO5


In [21]:
df_game = df_total.groupby(["player","match_type"]).apply(
    lambda group: pd.Series({
        "小局数" : group["Result"].count(),
        "小局胜率" : (group["Result"] == "Victory").mean()
    })
).reset_index()

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_30196\3708801551.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_game = df_total.groupby(["player","match_type"]).apply(


In [22]:
df_match = match_result.groupby(["player","match_type"]).apply(
    lambda group: pd.Series({
        "大局数": group["match_result"].count(),
        "大局胜率": (group["match_result"] == "Victory").mean()
    })
).reset_index()

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_30196\794358746.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_match = match_result.groupby(["player","match_type"]).apply(


In [23]:
df_winrate_summary = pd.merge(df_match,df_game,on = ["player","match_type"])

In [24]:
df_winrate_summary[["大局数","小局数"]] =  df_winrate_summary[["大局数","小局数"]].round(0)
df_winrate_summary[["大局胜率","小局胜率"]] =  df_winrate_summary[["大局胜率","小局胜率"]].round(2)

In [25]:
df_winrate_summary

,player,match_type,大局数,大局胜率,小局数,小局胜率
0,Crisp,BO1,3.0,0.33,3.0,0.33
1,Crisp,BO3,33.0,0.55,78.0,0.56
2,Crisp,BO5,12.0,0.58,48.0,0.44
3,Hang,BO1,2.0,1.00,2.0,1.00
4,Hang,BO3,25.0,0.72,63.0,0.63
5,Hang,BO5,5.0,0.20,20.0,0.40
6,ON,BO1,3.0,0.33,3.0,0.33
7,ON,BO3,33.0,0.88,79.0,0.77
8,ON,BO5,14.0,0.79,57.0,0.65
9,meiko,BO1,3.0,0.67,3.0,0.67


In [26]:
pivot_df = df_winrate_summary.pivot_table(index="player",columns="match_type")
pivot_df

大局数              大局胜率              小局数              小局胜率        \
match_type  BO1   BO3   BO5   BO1   BO3   BO5  BO1   BO3   BO5   BO1   BO3   
player                                                                       
Crisp       3.0  33.0  12.0  0.33  0.55  0.58  3.0  78.0  48.0  0.33  0.56   
Hang        2.0  25.0   5.0  1.00  0.72  0.20  2.0  63.0  20.0  1.00  0.63   
ON          3.0  33.0  14.0  0.33  0.88  0.79  3.0  79.0  57.0  0.33  0.77   
meiko       3.0  35.0  13.0  0.67  0.86  0.38  3.0  78.0  50.0  0.67  0.82   

                  
match_type   BO5  
player            
Crisp       0.44  
Hang        0.40  
ON          0.65  
meiko       0.48

In [27]:
import great_tables
from great_tables import GT, loc, style

In [28]:
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

In [29]:
pivot_df

,大局数_BO1,大局数_BO3,大局数_BO5,大局胜率_BO1,大局胜率_BO3,大局胜率_BO5,小局数_BO1,小局数_BO3,小局数_BO5,小局胜率_BO1,小局胜率_BO3,小局胜率_BO5
player,,,,,,,,,,,,
Crisp,3.0,33.0,12.0,0.33,0.55,0.58,3.0,78.0,48.0,0.33,0.56,0.44
Hang,2.0,25.0,5.0,1.00,0.72,0.20,2.0,63.0,20.0,1.00,0.63,0.40
ON,3.0,33.0,14.0,0.33,0.88,0.79,3.0,79.0,57.0,0.33,0.77,0.65
meiko,3.0,35.0,13.0,0.67,0.86,0.38,3.0,78.0,50.0,0.67,0.82,0.48


In [30]:
pivot_df = pivot_df.reset_index()

In [31]:
GT(pivot_df)

player,大局数_BO1,大局数_BO3,大局数_BO5,大局胜率_BO1,大局胜率_BO3,大局胜率_BO5,小局数_BO1,小局数_BO3,小局数_BO5,小局胜率_BO1,小局胜率_BO3,小局胜率_BO5
Crisp,3.0,33.0,12.0,0.33,0.55,0.58,3.0,78.0,48.0,0.33,0.56,0.44
Hang,2.0,25.0,5.0,1.0,0.72,0.2,2.0,63.0,20.0,1.0,0.63,0.4
ON,3.0,33.0,14.0,0.33,0.88,0.79,3.0,79.0,57.0,0.33,0.77,0.65
meiko,3.0,35.0,13.0,0.67,0.86,0.38,3.0,78.0,50.0,0.67,0.82,0.48


In [32]:
for i in pivot_df.columns:
    print(i)

player
大局数_BO1
大局数_BO3
大局数_BO5
大局胜率_BO1
大局胜率_BO3
大局胜率_BO5
小局数_BO1
小局数_BO3
小局数_BO5
小局胜率_BO1
小局胜率_BO3
小局胜率_BO5


In [33]:
import polars as pl
import polars.selectors as cs
from great_tables import GT, loc, style


bo1 = [col for col in pivot_df.columns if col.endswith("_BO1")]
bo3 = [col for col in pivot_df.columns if col.endswith("_BO3")]
bo5 = [col for col in pivot_df.columns if col.endswith("_BO5")]

# yo

great_table = (
    GT(pivot_df)
    .tab_header("不同赛制的场数胜率比较, 按赛制")
    .tab_spanner(label="BO1", columns=bo1)
    .tab_spanner(label="BO3", columns=bo3)
    .tab_spanner(label="BO5", columns=bo5)
    .cols_label(
        player = "选手",
        大局数_BO1 = "大局数",
        大局数_BO3 = "大局数",
        大局数_BO5 =  "大局数",
        大局胜率_BO1 = "大局胜率",
        大局胜率_BO3 = "大局胜率",
        大局胜率_BO5 = "大局胜率",
        小局数_BO1 = "小局数",
        小局数_BO3= "小局数",
        小局数_BO5= "小局数",
        小局胜率_BO1 = "小局胜率",
        小局胜率_BO3 = "小局胜率",
        小局胜率_BO5= "小局胜率",
    )
    # formatting ----
    .fmt_number(
        columns=[col for col in pivot_df.columns if "数" in col],
        compact=True,
        decimals=0,
        # n_sigfig=2,
    )
    .fmt_percent(columns=[col for col in pivot_df.columns if "率" in col], decimals=0)
    # style ----
    .tab_style(
        style=style.fill(color="floralwhite"),
        locations=loc.body(columns=bo1),
    )
    .tab_style(
        style=style.fill(color="aliceblue"),
        locations=loc.body(columns=bo3),
    )
    .tab_style(
        style=style.fill(color="lavenderblush"),
        locations=loc.body(columns=bo5),
    )
    .tab_style(
        style=style.fill(color="moccasin"),
        locations=[
            loc.body(columns="大局胜率_BO1", rows=[1]),
            loc.body(columns="小局胜率_BO1", rows=[1])
        ]
    )
    .tab_style(
        style=style.fill(color="lightblue"),
        locations=[
            loc.body(columns="大局胜率_BO3", rows=[2]),
            loc.body(columns="小局胜率_BO3", rows=[3])
        ]
    )
    .tab_style(
        style=style.fill(color="lightpink"),
        locations=[
            loc.body(columns="大局胜率_BO5", rows=[2]),
            loc.body(columns="小局胜率_BO5", rows=[2])
        ]
    ))


https://stackoverflow.com/questions/22408237/named-colors-in-matplotlib  
https://posit-dev.github.io/great-tables/get-started/basic-styling.html

In [34]:
great_table

GT(_tbl_data=  player  大局数_BO1  大局数_BO3  大局数_BO5  大局胜率_BO1  大局胜率_BO3  大局胜率_BO5  小局数_BO1  \
0  Crisp      3.0     33.0     12.0      0.33      0.55      0.58      3.0   
1   Hang      2.0     25.0      5.0      1.00      0.72      0.20      2.0   
2     ON      3.0     33.0     14.0      0.33      0.88      0.79      3.0   
3  meiko      3.0     35.0     13.0      0.67      0.86      0.38      3.0   

   小局数_BO3  小局数_BO5  小局胜率_BO1  小局胜率_BO3  小局胜率_BO5  
0     78.0     48.0      0.33      0.56      0.44  
1     63.0     20.0      1.00      0.63      0.40  
2     79.0     57.0      0.33      0.77      0.65  
3     78.0     50.0      0.67      0.82      0.48  , _body=<great_tables._gt_data.Body object at 0x000002369414D550>, _boxhead=Boxhead([ColInfo(var='player', type=<ColInfoTypeEnum.default: 1>, column_label='选手', column_align='left', column_width=None), ColInfo(var='大局数_BO1', type=<ColInfoTypeEnum.default: 1>, column_label='大局数', column_align='right', column_width=None), ColInfo(var='大局胜率_BO1', type=<ColInfoTypeEnum.default: 1>, column_label='大局胜率', column_align='right', column_width=None), ColInfo(var='小局数_BO1', type=<ColInfoTypeEnum.default: 1>, column_label='小局数', column_align='right', column_width=None), ColInfo(var='小局胜率_BO1', type=<ColInfoTypeEnum.default: 1>, column_label='小局胜率', column_align='right', column_width=None), ColInfo(var='大局数_BO3', type=<ColInfoTypeEnum.default: 1>, column_label='大局数', column_align='right', column_width=None), ColInfo(var='大局胜率_BO3', type=<ColInfoTypeEnum.default: 1>, column_label='大局胜率', column_align='right', column_width=None), ColInfo(var='小局数_BO3', type=<ColInfoTypeEnum.default: 1>, column_label='小局数', column_align='right', column_width=None), ColInfo(var='小局胜率_BO3', type=<ColInfoTypeEnum.default: 1>, column_label='小局胜率', column_align='right', column_width=None), ColInfo(var='大局数_BO5', type=<ColInfoTypeEnum.default: 1>, column_label='大局数', column_align='right', column_width=None), ColInfo(var='大局胜率_BO5', type=<ColInfoTypeEnum.default: 1>, column_label='大局胜率', column_align='right', column_width=None), ColInfo(var='小局数_BO5', type=<ColInfoTypeEnum.default: 1>, column_label='小局数', column_align='right', column_width=None), ColInfo(var='小局胜率_BO5', type=<ColInfoTypeEnum.default: 1>, column_label='小局胜率', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000023684481B20>, _spanners=Spanners([SpannerInfo(spanner_id='BO1', spanner_level=0, spanner_label='BO1', spanner_units=None, spanner_pattern=None, vars=['大局数_BO1', '大局胜率_BO1', '小局数_BO1', '小局胜率_BO1'], built=None), SpannerInfo(spanner_id='BO3', spanner_level=0, spanner_label='BO3', spanner_units=None, spanner_pattern=None, vars=['大局数_BO3', '大局胜率_BO3', '小局数_BO3', '小局胜率_BO3'], built=None), SpannerInfo(spanner_id='BO5', spanner_level=0, spanner_label='BO5', spanner_units=None, spanner_pattern=None, vars=['大局数_BO5', '大局胜率_BO5', '小局数_BO5', '小局胜率_BO5'], built=None)]), _heading=Heading(title='不同赛制的场数胜率比较, 按赛制', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['大局数_BO1', '大局胜率_BO1', '小局数_BO1', '小局胜率_BO1'], rows=None, mask=None), grpname=None, colname='大局数_BO1', rownum=0, colnum=None, styles=[CellStyleFill(color='floralwhite')]), StyleInfo(locname=LocBody(columns=['大局数_BO1', '大局胜率_BO1', '小局数_BO1', '小局胜率_BO1'], rows=None, mask=None), grpname=None, colname='大局数_BO1', rownum=1, colnum=None, styles=[CellStyleFill(color='floralwhite')]), StyleInfo(locname=LocBody(columns=['大局数_BO1', '大局胜率_BO1', '小局数_BO1', '小局胜率_BO1'], rows=None, mask=None), grpname=None, colname='大局数_BO1', rownum=2, colnum=None, styles=[CellStyleFill(color='floralwhite')]), StyleInfo(locname=LocBody(columns=['大局数_BO1', '大局胜率_BO1', '小局数_BO1', '小局胜率_BO1'], rows=None, mask=None), grpname=None, colname='大局数_BO1', rownum=3, colnum=None, styles=[CellStyleFill(color='floralwhite')]), StyleInfo(locname=LocBody(columns=['大局数_BO1', '大局胜率_BO1', '小局数_BO1', '小局胜率_BO1'], rows=None, mask=None), 

### 英雄池



In [35]:
champion_df = df_total.groupby(["player","Champion"]).apply(
    lambda group: pd.Series({
        "总局数": group["Result"].count(),
        "平均时长": group["duration_minutes"].mean().round(2),
        "平均KDA": round((group["K"].sum() + group["A"].sum()) / (group["D"].sum() if group["D"].sum() != 0 else 1), 2),
        "胜率" : (group["Result"] == "Victory").mean().round(2),
        "胜局": (group["Result"] == "Victory").sum(),
        "平均死亡数" : group["D"].mean().round(2),
        "平均K+A": (group["K"].mean() + group["A"].mean()).round(2),
        "总死亡数" : group["D"].sum(),
        "总K+A": (group["K"].sum() + group["A"].sum())
    })
).reset_index()

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_30196\1883169609.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  champion_df = df_total.groupby(["player","Champion"]).apply(


In [36]:
df_total.groupby("player")['Champion'].nunique()

player
Crisp    24
Hang     15
ON       22
meiko    22
Name: Champion, dtype: int64

In [37]:
champion_df[champion_df["总局数"] > 3].sort_values("平均时长",ascending=False).head(10)

,player,Champion,总局数,平均时长,平均KDA,胜率,胜局,平均死亡数,平均K+A,总死亡数,总K+A
11,Crisp,Milio,4.0,37.70,15.67,0.75,3.0,0.75,11.75,3.0,47.0
19,Crisp,Renata Glasc,11.0,35.90,3.09,0.36,4.0,3.00,9.27,33.0,102.0
28,Hang,Leona,10.0,35.42,2.51,0.80,8.0,3.50,8.80,35.0,88.0
24,Hang,Alistar,7.0,34.21,1.48,0.29,2.0,4.43,6.57,31.0,46.0
12,Crisp,Nami,5.0,33.65,3.93,0.60,3.0,2.80,11.00,14.0,55.0
53,ON,Nautilus,17.0,33.56,2.54,0.65,11.0,4.24,10.76,72.0,183.0
36,Hang,Rell,10.0,33.33,4.52,0.70,7.0,3.10,14.00,31.0,140.0
60,ON,Tahm Kench,4.0,32.90,2.25,0.75,3.0,3.00,6.75,12.0,27.0
52,ON,Nami,9.0,32.82,12.00,0.89,8.0,1.11,13.33,10.0,120.0
63,meiko,Braum,8.0,32.76,4.50,0.62,5.0,2.75,12.38,22.0,99.0


In [38]:
champion_df.head(4)

,player,Champion,总局数,平均时长,平均KDA,胜率,胜局,平均死亡数,平均K+A,总死亡数,总K+A
0,Crisp,Alistar,11.0,31.87,2.29,0.64,7.0,3.73,8.55,41.0,94.0
1,Crisp,Annie,1.0,35.03,1.00,0.00,0.0,3.00,3.00,3.0,3.0
2,Crisp,Ashe,3.0,31.38,16.00,0.67,2.0,0.67,10.67,2.0,32.0
3,Crisp,Bard,2.0,36.31,7.33,0.50,1.0,1.50,11.00,3.0,22.0


In [39]:
champion_pivot = pd.pivot_table(champion_df, index = "Champion", columns = "player", values = "总局数").reset_index().sort_values(by = "Crisp", ascending=False)

In [40]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = champion_pivot["Champion"]

fig = make_subplots(2, 2, specs=[[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Crisp', 'Hang',"ON","meiko"])
fig.add_trace(go.Pie(labels=labels, values=champion_pivot["Crisp"], scalegroup='one',
                     name="Crisp"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=champion_pivot["Hang"], scalegroup='one',
                     name="Hang"), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=champion_pivot["ON"], scalegroup='one',
                     name="ON"), 2, 1)
fig.add_trace(go.Pie(labels=labels, values=champion_pivot["meiko"], scalegroup='one',
                     name="meiko"), 2, 2)

fig.update_traces(hoverinfo='label+percent+value', textinfo='none')
fig.update_traces(textposition='inside', textinfo='percent+label')
symbol_template = "seaborn"
fig.update_layout(template=symbol_template)
fig.update_layout(
    height=600,
    width=600,
    title_text="英雄使用饼图",
    showlegend=False,
    margin=dict(t=50, l=50, r=50, b=50),  # 更紧凑的边距
    grid=dict(rows=2, columns=2),          # 明确 grid 设置
)
fig.show()

In [41]:
champion_df.head(5)

,player,Champion,总局数,平均时长,平均KDA,胜率,胜局,平均死亡数,平均K+A,总死亡数,总K+A
0,Crisp,Alistar,11.0,31.87,2.29,0.64,7.0,3.73,8.55,41.0,94.0
1,Crisp,Annie,1.0,35.03,1.00,0.00,0.0,3.00,3.00,3.0,3.0
2,Crisp,Ashe,3.0,31.38,16.00,0.67,2.0,0.67,10.67,2.0,32.0
3,Crisp,Bard,2.0,36.31,7.33,0.50,1.0,1.50,11.00,3.0,22.0
4,Crisp,Blitzcrank,1.0,44.23,2.50,1.00,1.0,6.00,15.00,6.0,15.0


In [42]:
champion_counts = champion_df['Champion'].value_counts()

champions_over_3 = champion_counts[champion_counts > 3].index

filtered_df = champion_df[champion_df['Champion'].isin(champions_over_3)]
filtered_df["胜率"] = filtered_df["胜率"].apply(lambda x: 0.01 if x == 0 else x)
filtered_df = filtered_df.sort_values(by = "总局数",ascending=False).sort_values(by = "player",ascending=True)
filtered_df.head(3)

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_30196\1577859508.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,player,Champion,总局数,平均时长,平均KDA,胜率,胜局,平均死亡数,平均K+A,总死亡数,总K+A
0,Crisp,Alistar,11.0,31.87,2.29,0.64,7.0,3.73,8.55,41.0,94.0
11,Crisp,Milio,4.0,37.70,15.67,0.75,3.0,0.75,11.75,3.0,47.0
13,Crisp,Nautilus,10.0,29.18,1.05,0.20,2.0,4.20,4.40,42.0,44.0


In [43]:
import plotly.express as px
fig = px.histogram(filtered_df, x="Champion", y="胜率",
             color= "player", barmode='group',
             height=400,opacity = 0.7)
symbol_template = "seaborn"
fig.update_layout(template=symbol_template)
fig.show()

In [44]:
champion_df

,player,Champion,总局数,平均时长,平均KDA,胜率,胜局,平均死亡数,平均K+A,总死亡数,总K+A
0,Crisp,Alistar,11.0,31.87,2.29,0.64,7.0,3.73,8.55,41.0,94.0
1,Crisp,Annie,1.0,35.03,1.00,0.00,0.0,3.00,3.00,3.0,3.0
2,Crisp,Ashe,3.0,31.38,16.00,0.67,2.0,0.67,10.67,2.0,32.0
3,Crisp,Bard,2.0,36.31,7.33,0.50,1.0,1.50,11.00,3.0,22.0
4,Crisp,Blitzcrank,1.0,44.23,2.50,1.00,1.0,6.00,15.00,6.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...
78,meiko,Rumble,3.0,31.56,4.25,0.67,2.0,2.67,11.33,8.0,34.0
79,meiko,Seraphine,1.0,30.77,1.67,0.00,0.0,3.00,5.00,3.0,5.0
80,meiko,Soraka,1.0,27.48,1.67,0.00,0.0,3.00,5.00,3.0,5.0
81,meiko,Tahm Kench,6.0,31.93,9.00,0.83,5.0,1.67,15.00,10.0,90.0


In [45]:

fig = px.scatter(champion_df, x="总局数", y="胜率",
             color='player', hover_data = ["Champion"],marginal_x="box",marginal_y="box", opacity= 0.5, size = '平均KDA')
symbol_template = "seaborn"
fig.update_layout(template=symbol_template)
fig.show()

In [46]:
champion_df

,player,Champion,总局数,平均时长,平均KDA,胜率,胜局,平均死亡数,平均K+A,总死亡数,总K+A
0,Crisp,Alistar,11.0,31.87,2.29,0.64,7.0,3.73,8.55,41.0,94.0
1,Crisp,Annie,1.0,35.03,1.00,0.00,0.0,3.00,3.00,3.0,3.0
2,Crisp,Ashe,3.0,31.38,16.00,0.67,2.0,0.67,10.67,2.0,32.0
3,Crisp,Bard,2.0,36.31,7.33,0.50,1.0,1.50,11.00,3.0,22.0
4,Crisp,Blitzcrank,1.0,44.23,2.50,1.00,1.0,6.00,15.00,6.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...
78,meiko,Rumble,3.0,31.56,4.25,0.67,2.0,2.67,11.33,8.0,34.0
79,meiko,Seraphine,1.0,30.77,1.67,0.00,0.0,3.00,5.00,3.0,5.0
80,meiko,Soraka,1.0,27.48,1.67,0.00,0.0,3.00,5.00,3.0,5.0
81,meiko,Tahm Kench,6.0,31.93,9.00,0.83,5.0,1.67,15.00,10.0,90.0


In [47]:
fig = px.scatter(filtered_df[filtered_df["总局数"]>1], x="平均KDA", y="胜率",
             color='player', hover_data = ["Champion"],marginal_x="box",marginal_y="box", opacity= 0.5)
symbol_template = "seaborn"
fig.update_layout(template=symbol_template)
fig.show()

### K+A vs D

In [48]:
df_total.head(3)

,Champion,Result,Date,Game,player,Tournament,duration_minutes,K,D,A,match_result,match_type
0,Maokai,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,27.750000,0,5,6,Defeat,BO5
1,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,31.616667,1,3,9,Defeat,BO5
2,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,39.283333,0,3,8,Defeat,BO5


In [49]:
df_total["K+A"] = df_total["K"] + df_total["A"]

In [50]:
fig = px.scatter(df_total, x="D", y="K+A",
             color='player', hover_data = ["player","Game","Champion"],marginal_x="box",marginal_y="box", opacity= 0.5, symbol = 'Result')
symbol_template = "seaborn"
fig.update_layout(template=symbol_template)
fig.show()

In [51]:
df_total.head(3)

,Champion,Result,Date,Game,player,Tournament,duration_minutes,K,D,A,match_result,match_type,K+A
0,Maokai,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,27.750000,0,5,6,Defeat,BO5,6
1,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,31.616667,1,3,9,Defeat,BO5,10
2,Renata Glasc,Defeat,2024-10-26,WBG vs BLG,Crisp,Worlds Main Event 2024,39.283333,0,3,8,Defeat,BO5,8


In [52]:
import plotly.express as px
fig = px.histogram(df_total, x="D", y = "K+A", color="player", marginal="rug",histfunc = "avg", # can be `box`, `violin`
                         hover_data=df_total.columns, opacity = 0.7, template="seaborn")
fig.update_layout(barmode="group")
fig.show()

In [53]:
fig = px.scatter(df_total, x="D", y = "K+A", color="player", marginal_y="box",  marginal_x="box", # can be `box`, `violin`
                         hover_data=df_total.columns, opacity = 0.5, trendline="ols", template="seaborn")
fig.show()

### date vs kda/winrate(xiaoju/daju)

In [54]:
import datetime
df_total[df_total["Date"].dt.year != 2024]


,Champion,Result,Date,Game,player,Tournament,duration_minutes,K,D,A,match_result,match_type,K+A


In [55]:
df_total["month"] = df_total["Date"].dt.month
df_total["KDA"] = df_total.apply(lambda row: (row["K"] + row["A"]) / (row["D"] if row["D"] != 0 else 1), axis=1)

In [56]:
df_total.sort_values(by="KDA")

,Champion,Result,Date,Game,player,Tournament,duration_minutes,K,D,A,match_result,match_type,K+A,month,KDA
354,Rell,Defeat,2024-10-19,Top Esports vs T1,meiko,Worlds Main Event 2024,33.750000,0,4,0,Defeat,BO5,0,10,0.0
251,Leona,Defeat,2024-07-26,NIP vs BLG,ON,LPL Summer Season 2024,28.200000,0,2,0,Victory,BO3,0,7,0.0
83,Nautilus,Defeat,2024-04-03,LNG Esports vs WBG,Crisp,LPL Spring Playoffs 2024,28.533333,0,5,0,Victory,BO5,0,4,0.0
416,Renata Glasc,Defeat,2024-05-14,Top Esports vs G2 Esports,meiko,MSI 2024,24.716667,0,5,0,Defeat,BO5,0,5,0.0
396,Braum,Defeat,2024-07-07,T1 vs Top Esports,meiko,Esports World Cup 2024,26.100000,0,5,1,Defeat,BO5,1,7,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,Milio,Victory,2024-01-26,BLG vs RNG,ON,LPL Spring 2024,31.916667,3,1,15,Victory,BO3,18,1,18.0
419,Ashe,Victory,2024-05-11,GEN vs Top Esports,meiko,MSI 2024,28.200000,3,0,16,Defeat,BO5,19,5,19.0
311,Renata Glasc,Victory,2024-04-11,BLG vs Top Esports,ON,LPL Spring Playoffs 2024,37.883333,1,1,19,Victory,BO5,20,4,20.0
152,Rell,Victory,2024-07-27,LNG Esports vs LGD Gaming,Hang,LPL Summer Season 2024,30.083333,1,0,20,Victory,BO3,21,7,21.0


In [57]:
df_total.Tournament.unique()

array(['Worlds Main Event 2024', 'LPL Regional Finals 2024',
       'LPL Summer Playoffs 2024', 'LPL Summer Season 2024',
       'LPL Summer Placements 2024', 'LPL Spring Playoffs 2024',
       'LPL Spring 2024', 'Esports World Cup 2024', 'MSI 2024'],
      dtype=object)

In [58]:
import plotly.graph_objects as go
fig = px.box(df_total, x="Tournament", y="KDA", color="player",template = "seaborn")
# fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_xaxes(categoryorder="array", categoryarray = df_total.Tournament.unique())

fig.show()

In [59]:
import plotly.figure_factory as ff

# Create a list of KDA arrays for each player
players = df_total["player"].unique()
kda_groups = [df_total[df_total["player"] == player]["KDA"].dropna().tolist() for player in players]

# Create the distplot
fig = ff.create_distplot(
    kda_groups,
    group_labels=players,
    show_hist=False,  # hide histogram if you just want smooth curves
    show_rug=False     # optional: small ticks on x-axis
)

fig.update_layout(template="seaborn", title="KDA Distribution per Player")
fig.show()